In [ ]:
from typing import Optional, cast
import pandas as pd
from yfinance3 import YFinance3
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import solara
import solara.express as solara_px  # similar to plotly express, but comes with cross filters
import solara.lab
from solara.components.columns import Columns
from solara.components.file_drop import FileDrop
import os
import requests
import json
import seaborn as sns
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
import csv
from scripts.funds import *
import dataclasses
from typing import Callable
import reacton.ipyvuetify as v
import solara
from solara.lab.toestand import Ref
from scripts.init_values import *
from scripts.stocks import *
from scripts.solara_ui import *

In [ ]:

button_style = 'font-size: 11px; border-radius: 3px; word-wrap: break-word;'

def columns_change(columns):
    df = Fund.df_full.value
    Fund.df.value = df[columns]

@solara.component
def toggle_panel(columns):

        with solara.ToggleButtonsMultiple(columns, on_value=columns_change, style="display: grid; grid-template-columns: 1fr 1fr 1fr 1fr 1fr; grid-column-gap: 2px; grid-row-gap: 2px;"):
                solara.Button("Symbol", value="Symbol", style=button_style, text=True)
                solara.Button("Name", value="Name", style=button_style, text=True)
                solara.Button("Sector", value="Sector", style=button_style, text=True)
                solara.Button("RSI", value="RSI_14", style=button_style, text=True)
                solara.Button("rsi slope", value="rsi_slope", style=button_style, text=True)
                solara.Button("SMA", value="SMA_20", style=button_style, text=True)
                solara.Button("EMA", value="EMA_50", style=button_style, text=True)
                solara.Button("ADX", value="ADX_14", style=button_style, text=True)
                solara.Button("DMP", value="DMP_14", style=button_style, text=True)
                solara.Button("DMN", value="DMN_14", style=button_style, text=True)
                solara.Button("adx slope", value="adx_slope", style=button_style, text=True)
                solara.Button("Day", value="Day", style=button_style, text=True)
                solara.Button("Week", value="Week", style=button_style, text=True)
                solara.Button("Month", value="Month", style=button_style, text=True)
                solara.Button("Last Close", value="Close", style=button_style, text=True)
                solara.Button("MACD", value="MACD_12_26_9", style=button_style, text=True)
                solara.Button("Histogram", value="MACDh_12_26_9", style=button_style, text=True)
                solara.Button("Signal", value="MACDs_12_26_9", style=button_style, text=True)
                solara.Button("BB Lower", value="BBL_5_2.0", style=button_style, text=True)
                solara.Button("BB Middle", value="BBM_5_2.0", style=button_style, text=True)
                solara.Button("BB Upper", value="BBU_5_2.0", style=button_style, text=True)
                



rank_value = solara.reactive("rsi")
float_value = solara.reactive(0)
temp_value = solara.reactive(0)

column_value = solara.reactive(["Symbol", "Name", "Sector", "RSI_14", "rsi_slope", "SMA_20", "EMA_50", "ADX_14", "DMP_14", "DMN_14", "adx_slope", "Day", "Week",
               "Month", "Close"]) 
column_list = ["Symbol", "Name", "Sector", "RSI_14", "rsi_slope", "SMA_20", "EMA_50", "ADX_14", "DMP_14", "DMN_14", "adx_slope", "Day", "Week",
               "Month", "Close", "MACD_12_26_9", "MACDh_12_26_9", "MACDs_12_26_9", "BBL_5_2.0", "BBM_5_2.0", "BBU_5_2.0", "BBB_5_2.0", "BBP_5_2.0"]

rank_values = ["rsi", "adx", "max_52_week"]



Stock.stock_init()
Fund.fund_init()

class State:
    rules = solara.reactive(init_rules_custom)
    
    start_point = solara.reactive(0)
    finish_point = solara.reactive(0)
    rank_point = solara.reactive(0)
    rank_value = solara.reactive(rank_values[0])
    
    t_start = solara.reactive(0)
    t_finish = solara.reactive(0)
    t_value = solara.reactive(0)
    
    indicator_weights = {
        "rsi": 0,
        "adx": 0,
        "max_52_week": 0,
    }
    
    
    def on_delete(item: RuleItem):
        new_items = list(State.rules.value)
        new_items.remove(item)
        State.rules.value = new_items
    
    def new_rule():
        State.rules.value = State.rules.value + [RuleItem(State.rank_value.value, State.start_point.value, State.finish_point.value, State.rank_point.value)]
        State.start_point.set(0)
        State.finish_point.set(0)
        State.rank_point.set(0)

    def rank_value_changed(new_value):
        State.rank_value.value = new_value
    
    def on_save(rule):
        print(rule)
        print(State.rules.value)
        
    def value_updated(new_value):
        print(new_value)
    
    @solara.component
    def RankListItem(rule):
        editing, set_editing = solara.use_state(False)
        
        State.t_start.value = rule.start
        State.t_finish.value = rule.finish
        State.t_value.value = rule.value
        
        with v.ListItem():

            solara.InputInt("From", value=rule.start, continuous_update=True)
            solara.InputInt("To", value=rule.finish, continuous_update=True)
            solara.InputInt("Rank", value=rule.value, continuous_update=True)

            
            solara.Column(children=[
                (
                    solara.IconButton(icon_name="save", on_click=lambda: set_editing(False))
                    if editing 
                    else 
                    solara.IconButton(icon_name="edit", on_click=lambda: set_editing(True))
                ),
                solara.IconButton(icon_name="delete", on_click=lambda: State.on_delete(rule))
            ])
            with v.Dialog(v_model=editing, persistent=True, max_width="500px", on_v_model=set_editing):
                if editing:  # 'reset' the component state on open/close

                    #def on_delete_in_edit():
                    #    on_delete(todo_item.value)
                    #    set_edit(False)

                    State.RuleEdit(rule, on_close=lambda: set_editing(False))

    @solara.component
    def RuleEdit(rule, on_close):
        
        with solara.Card("Edit", margin=0):
            
            solara.InputInt("From", value=rule.start, continuous_update=True)
            solara.InputInt("To", value=rule.finish, continuous_update=True)
            solara.InputInt("Rank", value=rule.value, continuous_update=True)
            
            with solara.CardActions():
                v.Spacer()
                solara.Button("Save", icon_name="mdi-content-save", on_click=State.on_save(rule), outlined=True, text=True)
                solara.Button("Close", icon_name="mdi-window-close", on_click=on_close, outlined=True, text=True)
                solara.Button("Delete", icon_name="mdi-delete", outlined=True, text=True)        
            

    def weight_changed(new_value):
        State.indicator_weights[State.rank_value.value] = new_value

end_date = datetime.today()
start_date = end_date - timedelta(days=350) 
    
tabs = ['custom', 'default', 'file']
tabs_value = solara.reactive("custom")

fund_ind = solara.reactive(0)
stock_ind = solara.reactive(0)

read_dict = {}
with open('funds_list.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        key, value = row[0], row[1].split(', ')
        read_dict[key] = [str(val) for val in value]


def ruleItem_to_dict(rule):
    # convert rule item list to classic python dict
    rule_dict = {}
    rule_dict['rsi'] = {}
    rule_dict['adx'] = {}
    rule_dict['max_52_week'] = {}

    for item in rule:
        rule_dict[item.indicator][(item.start, item.finish)] = item.value

    rule_dict['RSI_14'] = rule_dict.pop('rsi')
    rule_dict['ADX_14'] = rule_dict.pop('adx')
    rule_dict['52_weeks'] = rule_dict.pop('max_52_week')
        
    return rule_dict

rank_options=['RSI_14', 'ADX_14', '52_weeks']

def weights_convert(weights):
    weights['RSI_14'] = weights.pop('rsi')
    weights['ADX_14'] = weights.pop('adx')
    weights['52_weeks'] = weights.pop('max_52_week')
    return weights
        

def apply_filter(input_value, param, rules, weights):
    
    print(rules)
    
    for interval, value in rules[param].items():
        
        print(input_value)
        
        if interval[0] <= input_value < interval[1]:
            return value * weights[param]
    return 0

def calc_rank():

    if Fund.df.value is not None:
            rule_dict = ruleItem_to_dict(State.rules.value)
            weights = weights_convert(State.indicator_weights)
            
            temp_value = Fund.df_full.value
            
            for val in rank_options:
                
                temp_value[str(val)+"_rank"] = temp_value[val].apply(apply_filter, args=(val, rule_dict, weights))

            temp_value["Rank"] = temp_value[rank_options[0]+"_rank"] + temp_value[rank_options[1]+"_rank"] + temp_value[rank_options[2]+"_rank"]

            Fund.df.value = temp_value
            
            print(temp_value["Rank"])

        
@solara.component
def Page():     

        
    def on_action_cell(column, row_index):
        with Columns(widths=[2, 4]):
            solara_px.scatter(
                    df,
                    State.x.value,
                    State.y.value,
                    size=State.size.value,
                    color=State.color.value,
                    size_max=State.size_max.value,
                    log_x=State.logx.value,
                    log_y=State.logy.value,
                )
    
    def update_data():
        fund_len = len(read_dict.keys())
        for i, fundname in enumerate(read_dict.keys()):

            last_date = pd.DataFrame()
            fund_ind.value = int(round((i+1)/fund_len*100, 0))

            tick_len = len(read_dict[fundname])

            for j, tick in enumerate(read_dict[fundname]):

                stock_ind.value = int(round((j+1)/tick_len*100, 0))

                stock_data = yf.download(tick, start=start_date, end=end_date)
                stock_data.ta.macd(append=True)
                stock_data.ta.rsi(append=True)
                stock_data.ta.bbands(append=True)
                stock_data.ta.sma(length=20, append=True)
                stock_data.ta.ema(length=50, append=True)
                stock_data.ta.adx(append=True)
                stock_data.dropna(inplace=True)
                stock_data = stock_data.drop(columns=["Open", "Adj Close", "Volume"])

                max_value = stock_data['High'].max()
                min_value = stock_data['Low'].min()
                stock_data["max_52_week"] = (stock_data['Close'] - min_value)/(max_value-min_value)

                df = pd.DataFrame(stock_data.iloc[-1]).T
                df['Symbol'] = tick
                df['Day'] = (stock_data.iloc[-1]['Close'] - stock_data.iloc[-2]['Close']) / stock_data.iloc[-2]['Close'] * 252
                df['Week'] = (stock_data.iloc[-1]['Close'] - stock_data.iloc[-6]['Close']) / stock_data.iloc[-6]['Close'] * 50
                df['Month'] = (stock_data.iloc[-1]['Close'] - stock_data.iloc[-22]['Close']) / stock_data.iloc[-22]['Close'] * 12

                sliced = stock_data.iloc[-5:]

                rsi_sliced = linregress(range(len(sliced)), sliced['RSI_14'])
                adx_sliced = linregress(range(len(sliced)), sliced['ADX_14'])

                df['adx_slope'] = adx_sliced.slope
                df['rsi_slope'] = rsi_sliced.slope        

                if stock_data.iloc[-1]['Close'] > stock_data.iloc[-1]['SMA_20']:
                    if stock_data.iloc[-2]['Close'] < stock_data.iloc[-2]['SMA_20']:
                        sma_status = "Crossed up"
                    else:
                        sma_status = "More"
                else:
                    if stock_data.iloc[-2]['Close'] > stock_data.iloc[-2]['SMA_20']:
                        sma_status = "Crossed down"
                    else:
                        sma_status = "Less"


                if stock_data.iloc[-1]['MACD_12_26_9'] > stock_data.iloc[-1]['MACDs_12_26_9']:
                    if stock_data.iloc[-2]['MACD_12_26_9'] < stock_data.iloc[-2]['MACDs_12_26_9']:
                        macd_status = "Crossed up"
                    else:
                        macd_status = "More"
                else:
                    if stock_data.iloc[-2]['MACD_12_26_9'] > stock_data.iloc[-2]['MACDs_12_26_9']:
                        macd_status = "Crossed down"
                    else:
                        macd_status = "Less"

                if stock_data.iloc[-1]['Close'] > stock_data.iloc[-1]['BBU_5_2.0']:
                    bb_status = 'More'
                elif stock_data.iloc[-1]['Close'] < stock_data.iloc[-1]['BBL_5_2.0']:
                    bb_status = 'Less'
                else:
                    bb_status = 'Inside'


                df['sma_status'] = sma_status
                df['macd_status'] = macd_status
                df['bb_status'] = bb_status

                last_date = pd.concat([last_date, df], ignore_index=True)

                filepath = "indicators/" + tick + ".csv"        
                stock_data.to_csv(filepath)

            last_column = last_date.columns[-1]
            last_date = pd.concat([last_date[last_column], last_date.drop(columns=[last_column])], axis=1)
            last_date.to_csv("funds/" + fundname + ".csv", index=False)

      
    def get_button_color(label):
        if label == 'Somewhat-Bullish':
            return "light-green"
        if label == 'Neutral':
            return "light-grey"
        if label == "Bullish":
            return "green"
        if label == 'Somewhat-Bearish':
            return "red"
        if label == 'Bearish':
            return "red"
    

    
    def ranking_changed(new_value):
        if new_value == 'custom':
            State.rules.value = init_rules_custom
        elif new_value == 'default':
            State.rules.value = init_rules_default
    
    df = Fund.df.value
    mean_value = Fund.mean_value.value
    
    tick_df_full = Stock.tick_df.value
    info = Stock.info.value


    column_actions = [solara.ColumnAction(icon="mdi-arrow-down", name="From high to low", on_click=Fund.on_action_column_high),
                     solara.ColumnAction(icon="mdi-arrow-up", name="From low to high", on_click=Fund.on_action_column_low)]
    cell_actions = [solara.CellAction(icon="mdi-white-balance-sunny", name="User cell action", on_click=on_action_cell)]
    
    with solara.Sidebar():
        with solara.lab.Tabs():
            with solara.lab.Tab("Fund"):
    
                with solara.Card("Select fund", margin=0, elevation=0):
                    with solara.Column():
                        #with solara.Row():
                            solara.Select(label="Fund name", value=Fund.fund_value, values=list(read_dict.keys()))
                        #if df is not None:
                            solara.Markdown("**Filters:**")
                            solara.SliderRangeInt("RSI range", value=Fund.rsi_range, min=0, max=100)
                            solara.SliderRangeInt("ADX range", value=Fund.adx_range, min=0, max=100)
                            solara.Checkbox(label='ADX goes up', value=Fund.adx_up)

                            solara.Select(label='MACD vs SIGNAL', value=Fund.macd_range, values=['None', 'More', 'Less', 'Crossed up', 'Crossed down'])
                            solara.Select(label='СLOSE vs EMA(20)', value=Fund.close_ema, values=['None', 'More', 'Less', 'Crossed up', 'Crossed down'])
                            solara.Select(label='СLOSE vs BB', value=Fund.close_bb, values=['None', 'More', 'Less', 'Inside'])
                            
                            solara.Button("Apply filters", icon_name="mdi-filter-plus", outlined=False, on_click=Fund.load_fund_values)
                            
                            solara.Markdown("**Columns:**")
                            
                            toggle_panel(column_value)
                            
            with solara.lab.Tab("Update"):
                with solara.Card("Update data:", margin=0, elevation=0):
                    with solara.HBox():
                        solara.Markdown("**Last updated:** {}".format(last_date), style="margin-top:5px;")
                        solara.Button("Re-new", icon_name="mdi-refresh-circle", style="margin-left:20px;", on_click=update_data, outlined=False, color="green")
                    
                    solara.ProgressLinear(value=fund_ind.value, color="purple", style="height:5px;")
                    solara.ProgressLinear(value=stock_ind.value, color="blue", style="height:5px; margin-top:5px;")
            
            with solara.lab.Tab("Ranking"):
                if df is not None:
                    solara.ToggleButtonsSingle(value=tabs_value, values=tabs, on_value=ranking_changed)

                    with solara.Card(tabs_value.value, margin=0, elevation=0):

                        solara.ToggleButtonsSingle(value=State.rank_value, values=rank_values, on_value=State.rank_value_changed)
                        temp_value.value = State.indicator_weights[State.rank_value.value]
                        solara.SliderInt("Indicator weight:", value=temp_value, on_value=State.weight_changed, min=-10, max=10)
                        solara.Markdown("**Current rules:**")

                        for index, item in enumerate(State.rules.value):
                            if item.indicator == State.rank_value.value:
                                State.RankListItem(item)

                        solara.Markdown("**New rule:**")

                        with solara.HBox():
                            solara.InputInt("From", value=State.start_point)
                            solara.InputInt("To", value=State.finish_point)
                            solara.InputInt("Rank", value=State.rank_point)
                            solara.Button("Add", on_click=State.new_rule, classes=["primary", "add-button"])


                        solara.Button("Calculate rank", on_click=calc_rank, style='margin-top: 20px; text-align: center;')


    if df is not None:
        with solara.Columns([4, 1]):
            with solara.VBox():
                with solara.Card(title='Today stock analysis'):
                        
                        solara_df = solara.DataFrame(df, items_per_page=25, scrollable=True, column_actions=column_actions, cell_actions=cell_actions)
                
                with solara.Card(title='Heat map'):
                        
                        if mean_value is not None:
                            fig, ax = plt.subplots(figsize=(10, 6))
                            sns.heatmap(mean_value, cmap="YlGnBu", annot=True)
                            ax.set_title('Profit')
                            solara.FigureMatplotlib(fig)
                        
            #menu for top winners and lossers
            win_and_los_ui(df)
            
    else:
        solara.Info("Fund list not selected")


    # Top menu that choose stock parameters to show   
    Stock.stock_menu()
    

    if tick_df_full is not None:
        if not Stock.wrong_ticker.value:
            fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[3, 1, 1, 1])
            
            tick_df = Stock.slice_period(tick_df_full)
            
            if Stock.bb_box.value:
                fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df["BBL_5_2.0"], mode='lines', name='BBL', line = dict(color = 'yellow', width=1)), row=1, col=1)
                fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df["BBU_5_2.0"], mode='lines', name='BBU', line = dict(color = 'yellow', width=1), fillcolor='rgba(204,204,0,0.2)', fill='tonexty'), row=1, col=1)
            
            fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df['SMA_20'], mode='lines', name='SMA', line = dict(color = 'orange', width=1)), row=1, col=1)
            fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df['EMA_50'], mode='lines', name='EMA', line = dict(color = 'purple', width=1, shape = 'spline', smoothing = 1.3)), row=1, col=1)
            fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df['Close'], mode='lines', name='CLOSE', line = dict(color = 'black', width=3)), row=1, col=1)

            row_counter = 2
            if Stock.rsi_box.value:
                fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df["RSI_14"], mode='lines', name='RSI'), row=row_counter, col=1)
                row_counter += 1
                
            if Stock.macd_box.value:
                fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df["MACDs_12_26_9"], mode='lines', name='SIGNAL', line = dict(color = 'orange', width=2)), row=row_counter, col=1)
                fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df["MACD_12_26_9"], mode='lines', name='MACD', line = dict(color = 'purple', width=2)), row=row_counter, col=1)
                colors = ['green' if val >= 0 else 'red' for val in tick_df['MACDh_12_26_9']]
                fig.add_trace(go.Bar(x=tick_df['Date'], y=tick_df['MACDh_12_26_9'], name="HIST", marker_color=colors), row=row_counter, col=1)
                row_counter += 1
                
            if Stock.adx_box.value:
                fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df["ADX_14"], mode='lines', name='ADX', line = dict(color = 'black', width=2)), row=row_counter, col=1)
                fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df["DMP_14"], mode='lines', name='DX+', line = dict(color = 'green', width=2)), row=row_counter, col=1)
                fig.add_trace(go.Scatter(x=tick_df['Date'], y=tick_df["DMN_14"], mode='lines', name='DX-', line = dict(color = 'red', width=2)), row=row_counter, col=1)
            fig.update_layout(height=1200, width=1800)

            solara.FigurePlotly(fig)
            
            with solara.Columns([1, 1]):
                with solara.Card(title="Fundamental Info", subtitle=Stock.text.value):

                    solara.Markdown(
                        Stock.info.value
                    )

                with solara.VBox():
                    if False:
                        try:
                            url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={}&apikey=CYDTFOE3Z5T21F8G'.format(text.value)
                            r = requests.get(url)
                            news_data = r.json()

                            for i_news in range(6):    

                                timestamp_str = news_data['feed'][i_news]['time_published']
                                datetime_object = datetime.strptime(timestamp_str, '%Y%m%dT%H%M%S')
                                formatted_str = datetime_object.strftime('%Y-%m-%d %H:%M:%S')


                                with solara.Card(title=news_data['feed'][i_news]['title'], subtitle=formatted_str):
                                    solara.Markdown(
                                        news_data['feed'][i_news]['summary']
                                    )

                                    for sentiment in news_data['feed'][i_news]['ticker_sentiment']:

                                        button_color = get_button_color(sentiment['ticker_sentiment_label'])
                                        solara.Button(label = sentiment['ticker'], style="margin: 5px;", color=button_color, text=False)
                        except:
                            solara.Error("Cannot get the company news", icon=True)

                                
                                
                                
        else:
            solara.Error("You set the wrong sticker name", icon=True)

                
@solara.component
def Layout(children):
    route, routes = solara.use_route()
    return solara.AppLayout(children=children)



Page()